In [17]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Perceptron

from Classificadores import ELM, RBF, ELMHebbiano, run_cross_validation_on_elms, run_cross_validation_on_perceptron, getMeanAUC, getMeanAUC_Perceptron

In [2]:
# Importação da classificação "labels" (para todo o conjunto de dados, treinamento e teste)
data = pd.read_csv('./datasets/banknote_authentication/data_banknote_authentication.txt')
data.head()


,variance,skewness,curtosis,entropy,class
0,3.62160,8.6661,-2.8073,-0.44699,0
1,4.54590,8.1674,-2.4586,-1.46210,0
2,3.86600,-2.6383,1.9242,0.10645,0
3,3.45660,9.5228,-4.0112,-3.59440,0
4,0.32924,-4.4552,4.5718,-0.98880,0


In [3]:
# data information
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1372 entries, 0 to 1371
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   variance  1372 non-null   float64
 1   skewness  1372 non-null   float64
 2   curtosis  1372 non-null   float64
 3   entropy   1372 non-null   float64
 4   class     1372 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 53.7 KB


In [4]:
# Statistical description
data.describe()

,variance,skewness,curtosis,entropy,class
count,1372.000000,1372.000000,1372.000000,1372.000000,1372.000000
mean,0.433735,1.922353,1.397627,-1.191657,0.444606
std,2.842763,5.869047,4.310030,2.101013,0.497103
min,-7.042100,-13.773100,-5.286100,-8.548200,0.000000
25%,-1.773000,-1.708200,-1.574975,-2.413450,0.000000
50%,0.496180,2.319650,0.616630,-0.586650,0.000000
75%,2.821475,6.814625,3.179250,0.394810,1.000000
max,6.824800,12.951600,17.927400,2.449500,1.000000


In [5]:
# Extrai rótulos y dos atributos X

y = data['class']
y = np.array(y.replace({0:-1}))

X = np.array(data.drop(['class'], axis=1))

In [6]:
# Divide os atributos em conjunto de treinamento e de testes na razaõ 75%/ 25%
X_train, X_test, y_train, y_test = train_test_split(X, y) 

In [7]:
# Aplica a mesma escala para ambos os datasets
scaler = StandardScaler()
X_train_scl = scaler.fit_transform(X_train)
X_test_scl = scaler.transform(X_test) # note that we transform rather than fit_transform

### ELM

In [37]:
NNeuronios = [2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]
lambdas = [0, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]

cv_scores_mean, cv_scores_std = run_cross_validation_on_elms(X_train_scl, y_train, NNeuronios, lambdas, ELM)

In [41]:
idx_bestNNeuronios , idx_bestLambda = np.unravel_index(np.argmax(cv_scores_mean, axis=None), cv_scores_mean.shape) 
best_Nneuronios = NNeuronios[idx_bestNNeuronios]
best_lambda = lambdas[idx_bestLambda]
best_score = cv_scores_mean[idx_bestNNeuronios][idx_bestLambda]
best_score_std = cv_scores_std[idx_bestNNeuronios][idx_bestLambda]

print('Ao utilizar {} neurônios na camada intermediária e fator de regularização igual à {} foi atingindo a melhor acurácia média de validação cruzada sobre o conjunto de treinamento: {} +/- {}%'.format(best_Nneuronios, best_lambda, round(best_score*100,3), round(best_score_std*100, 3)))

Ao utilizar 64 neurônios na camada intermediária e fator de regularização igual à 0.0001 foi atingindo a melhor acurácia média de validação cruzada sobre o conjunto de treinamento: 100.0 +/- 0.0%


In [42]:
meanAUC, stdAUC = getMeanAUC(ELM, X_train_scl, y_train, X_test_scl, y_test, best_Nneuronios, best_lambda)

print('AUC médio e desvio padrão após dez tentativas {} +/- {}'.format(round(meanAUC,3), round(stdAUC,3)))

AUC médio e desvio padrão após dez tentativas 1.0 +/- 0.0


### RBF

In [64]:
parameters = {'p':NNeuronios, 'lambdar':lambdas}
rbf = RBF(p = 2)

clf = GridSearchCV(rbf, parameters, scoring='accuracy', n_jobs = -1, cv = 10)
clf.fit(X_train_scl, y_train)

GridSearchCV(cv=10, estimator=RBF(p=2), n_jobs=-1,
             param_grid={'lambdar': [0, 1000, 10000, 100000],
                         'p': [2, 8, 16, 32, 64, 128]},
             scoring='accuracy')

In [78]:
idx = np.where(clf.cv_results_['mean_test_score'] == np.amax(clf.cv_results_['mean_test_score']))[0][0]
best_score = clf.cv_results_['mean_test_score'][idx]
best_score_std = clf.cv_results_['std_test_score'][idx]

best_Nneuronios = clf.best_params_['p']
best_lambda = clf.best_params_['lambdar']

print('Ao utilizar {} neurônios na camada intermediária e fator de regularização igual à {} foi atingindo a melhor acurácia média de validação cruzada sobre o conjunto de treinamento: {} +/- {}%'.format(best_Nneuronios, best_lambda, round(best_score*100,3), round(best_score_std*100, 3)))

Ao utilizar 64 neurônios na camada intermediária e fator de regularização igual à 10000 foi atingindo a melhor acurácia média de validação cruzada sobre o conjunto de treinamento: 96.988 +/- 1.402%


In [79]:
meanAUC, stdAUC = getMeanAUC(RBF, X_train_scl, y_train, X_test_scl, y_test, best_Nneuronios, best_lambda)

print('AUC médio e desvio padrão após dez tentativas {} +/- {}'.format(round(meanAUC,3), round(stdAUC,3)))

AUC médio e desvio padrão após dez tentativas 0.97 +/- 0.013


### Hebbiano

In [10]:
NNeuronios = [2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]
lambdas = [0, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]

parameters = {'p':NNeuronios, 'lambdar':lambdas}
heb = ELMHebbiano()

clf = GridSearchCV(heb, parameters, scoring='accuracy', n_jobs = -1, cv = 10)
clf.fit(X_train_scl, y_train)

GridSearchCV(cv=10, estimator=ELMHebbiano(), n_jobs=-1,
             param_grid={'lambdar': [0, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100,
                                     1000, 10000],
                         'p': [2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]},
             scoring='accuracy')

In [11]:
idx = np.where(clf.cv_results_['mean_test_score'] == np.amax(clf.cv_results_['mean_test_score']))[0][0]
best_score = clf.cv_results_['mean_test_score'][idx]
best_score_std = clf.cv_results_['std_test_score'][idx]

best_Nneuronios = clf.best_params_['p']
best_lambda = clf.best_params_['lambdar']

print('Ao utilizar {} neurônios na camada intermediária e fator de regularização igual à {} foi atingindo a melhor acurácia média de validação cruzada sobre o conjunto de treinamento: {} +/- {}%'.format(best_Nneuronios, best_lambda, round(best_score*100,3), round(best_score_std*100, 3)))

Ao utilizar 64 neurônios na camada intermediária e fator de regularização igual à 1 foi atingindo a melhor acurácia média de validação cruzada sobre o conjunto de treinamento: 86.981 +/- 3.47%


In [14]:
meanAUC, stdAUC = getMeanAUC(ELMHebbiano, X_train_scl, y_train, X_test_scl, y_test, best_Nneuronios, best_lambda)

print('AUC médio e desvio padrão após dez tentativas {} +/- {}'.format(round(meanAUC,3), round(stdAUC,3)))

AUC médio e desvio padrão após dez tentativas 0.836 +/- 0.03


### Perceptron

In [18]:
lambdas = [0, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]

parameters = {'alpha': lambdas}
model = Perceptron(penalty='l2')

clf = GridSearchCV(model, parameters, scoring='accuracy', n_jobs = -1, cv = 10)
clf.fit(X_train_scl, y_train)

GridSearchCV(cv=10, estimator=Perceptron(penalty='l2'), n_jobs=-1,
             param_grid={'alpha': [0, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100,
                                   1000, 10000]},
             scoring='accuracy')

In [19]:
idx = np.where(clf.cv_results_['mean_test_score'] == np.amax(clf.cv_results_['mean_test_score']))[0][0]
best_score = clf.cv_results_['mean_test_score'][idx]
best_score_std = clf.cv_results_['std_test_score'][idx]

best_lambda = clf.best_params_['alpha']

print('Ao utilizar fator de regularização igual à {} foi atingindo a melhor acurácia média de validação cruzada sobre o conjunto de treinamento: {} +/- {}%'.format(best_lambda, round(best_score*100,3), round(best_score_std*100, 3)))

Ao utilizar fator de regularização igual à 0 foi atingindo a melhor acurácia média de validação cruzada sobre o conjunto de treinamento: 98.153 +/- 1.103%


In [21]:
meanAUC, stdAUC = getMeanAUC_Perceptron(X_train_scl, y_train, X_test_scl, y_test, best_lambda)

print('AUC médio e desvio padrão após dez tentativas {} +/- {}'.format(round(meanAUC,3), round(stdAUC,3)))

AUC médio e desvio padrão após dez tentativas 0.985 +/- 0.0
